# Instalações

In [9]:
import pandas as pd
import numpy as np
import gdown
import requests
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

In [2]:
arquivo_destino_base = "dataset_{}.csv"

id = {
    "cartao": "1nriPPuYUMXeB6BkCjz_bQI_45WZfxViC"
}

dataframes = {}

for key, file_id in id.items():
    url = f"https://drive.google.com/uc?id={file_id}"
    arquivo_destino = arquivo_destino_base.format(key)

    # Baixar o arquivo
    gdown.download(url, arquivo_destino, quiet=False)

    # Ler o arquivo CSV
    df = pd.read_csv(arquivo_destino, sep=",")
    dataframes[key] = df

print(dataframes)

Downloading...
From (original): https://drive.google.com/uc?id=1nriPPuYUMXeB6BkCjz_bQI_45WZfxViC
From (redirected): https://drive.google.com/uc?id=1nriPPuYUMXeB6BkCjz_bQI_45WZfxViC&confirm=t&uuid=071203f4-d2e3-4873-8d4f-5b4f881aa681
To: /content/dataset_cartao.csv
100%|██████████| 151M/151M [00:03<00:00, 48.9MB/s]


{'cartao':             Time         V1         V2        V3        V4        V5  \
0            0.0  -1.359807  -0.072781  2.536347  1.378155 -0.338321   
1            0.0   1.191857   0.266151  0.166480  0.448154  0.060018   
2            1.0  -1.358354  -1.340163  1.773209  0.379780 -0.503198   
3            1.0  -0.966272  -0.185226  1.792993 -0.863291 -0.010309   
4            2.0  -1.158233   0.877737  1.548718  0.403034 -0.407193   
...          ...        ...        ...       ...       ...       ...   
284802  172786.0 -11.881118  10.071785 -9.834783 -2.066656 -5.364473   
284803  172787.0  -0.732789  -0.055080  2.035030 -0.738589  0.868229   
284804  172788.0   1.919565  -0.301254 -3.249640 -0.557828  2.630515   
284805  172788.0  -0.240440   0.530483  0.702510  0.689799 -0.377961   
284806  172792.0  -0.533413  -0.189733  0.703337 -0.506271 -0.012546   

              V6        V7        V8        V9  ...       V21       V22  \
0       0.462388  0.239599  0.098698  0.363787  .

# Entendimento dos dados

A proporção das classes indica um forte desbalanceamento no conjunto de dados, com a classe "0" (transações legítimas) representando a maioria absoluta das amostras e a classe "1" (transações fraudulentas) sendo uma minoria muito pequena. Especificamente, existem 284.315 amostras da classe "0" e apenas 492 da classe "1".

In [3]:
# Carregando os dados
data = pd.read_csv('dataset_cartao.csv')

# Visualização básica dos dados
print(data.head())

# Verificando a proporção de classes
print(data['Class'].value_counts())


   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28 

# Aplicação de técnica de oversampling

Esse desbalanceamento pode causar problemas para o modelo de machine learning, como a tendência de prever sempre a classe majoritária ("0"), resultando em uma alta acurácia, mas desempenho ruim em termos de recall e F1-score para a classe minoritária ("1").

Por isso, utilizaremos a técnica de oversampling (aumentar o número de exemplos da classe minoritária) SMOTE (Synthetic Minority Over-sampling Technique).

In [6]:
# Definindo X e y
X = data.drop(columns=['Class'])
y = data['Class']

# Aplicando SMOTE para balancear as classes
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

# Verificando a nova proporção de classes
print(pd.Series(y_res).value_counts())

# Dividindo os dados balanceados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

Class
0    284315
1    284315
Name: count, dtype: int64


# Treinamento do modelo
O treinamento de um modelo de machine learning é o processo pelo qual o modelo "aprende" a realizar previsões ou classificações com base em um conjunto de dados de entrada.
Dado o tempo excessivo necessário para treinar o modelo com todo o conjunto de dados e a desbalanceamento extremo entre as classes, reduzi o tamanho da amostra para 1% dos dados para acelerar o processo de treinamento e avaliação. No entanto, mesmo com essa amostra menor, o modelo não foi capaz de aprender de forma eficaz devido à natureza altamente desbalanceada dos dados.

- ANÁLISE DE RESULTADOS:

Accuracy (0.9977): A acurácia está muito alta, mas isso não significa que o modelo é bom. Isso ocorre porque a classe de fraudes (classe 1) é extremamente rara, e o modelo está basicamente prevendo a classe majoritária (classe 0) quase o tempo todo, resultando em uma alta acurácia.

Precision (0.0000): A precisão é 0, o que indica que o modelo não está identificando nenhum verdadeiro positivo. Ou seja, das poucas vezes em que o modelo prevê uma fraude (classe 1), todas são falsos positivos.

Recall (0.0000): O recall também é 0, indicando que o modelo não consegue identificar nenhuma fraude real entre os dados de teste. Isso é preocupante em aplicações de detecção de fraudes, onde a capacidade de capturar fraudes é crucial.

F1-Score (0.0000): Como tanto a precisão quanto o recall são 0, o F1-Score, que é a média harmônica dessas duas métricas, também é 0. Isso confirma que o modelo é ineficaz na detecção de fraudes.

AUC-ROC (0.4801): O AUC-ROC está próximo de 0,5, o que indica que o modelo está praticamente fazendo previsões aleatórias. Um modelo AUC-ROC de 0,5 sugere que ele não consegue distinguir entre as classes de forma alguma.

In [10]:
X_sample, _, y_sample, _ = train_test_split(X, y, train_size=0.01, random_state=42, stratify=y)

# Dividindo a amostra em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.3, random_state=42)

# Definindo e treinando o modelo de rede neural simples
model = MLPClassifier(hidden_layer_sizes=(50,), max_iter=50, random_state=42)
model.fit(X_train, y_train)

# Prevendo no conjunto de teste
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probabilidades para calcular o AUC-ROC

# Avaliando o modelo
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_pred_proba)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"AUC-ROC: {auc_roc:.4f}")

Accuracy: 0.9977
Precision: 0.0000
Recall: 0.0000
F1-Score: 0.0000
AUC-ROC: 0.4801


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Ajuste de hiperparâmetros
Ajustar os hiperparâmetros é uma etapa crucial para otimizar o desempenho de um modelo de machine learning.


## Grid Search
O Grid Search envolve a definição de uma grade (grid) de valores para cada hiperparâmetro e, em seguida, a execução de treinamentos exaustivos para todas as combinações possíveis desses valores.

Os resultados do Grid Search indicam que a melhor combinação de hiperparâmetros para o modelo foi:

Ativação: tanh
Alpha: 0.0001
Hidden Layer Sizes: (50,)
Learning Rate: constant
Solver: sgd

No entanto, o F1 Score resultante foi 0.0000, o que sugere que o modelo não conseguiu capturar nenhuma instância positiva corretamente. Isso é um indicativo de que o modelo ainda está severamente desequilibrado ou subajustado (underfitting).


In [11]:
# Definindo o modelo básico
model = MLPClassifier(max_iter=100)

# Definindo a grade de hiperparâmetros
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant', 'adaptive'],
}

# Configurando o GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, scoring='f1')

# Executando o Grid Search
grid_search.fit(X_train, y_train)

# Melhores hiperparâmetros encontrados
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best parameters found: {best_params}")
print(f"Best F1 Score: {best_score:.4f}")


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Best parameters found: {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50,), 'learning_rate': 'constant', 'solver': 'sgd'}
Best F1 Score: 0.0000


## Random Search
O Random Search, por outro lado, seleciona aleatoriamente combinações de valores para os hiperparâmetros a partir de uma distribuição predefinida. Embora não explore todas as combinações possíveis, ele pode ser significativamente mais rápido, especialmente em casos onde o espaço de hiperparâmetros é muito grande.

Os resultados do RandomizedSearchCV para o modelo otimizado são os seguintes:

Accuracy: 0.9977
Precision: 0.0000
Recall: 0.0000
F1-Score: 0.0000
AUC-ROC: 0.4918

A acurácia do modelo é muito alta, indicando que o modelo acerta uma grande parte das previsões, mas isso pode ser enganoso em um dataset altamente desbalanceado. Neste caso, a alta acurácia pode estar sendo dominada pelas previsões corretas da classe majoritária (classe 0).

Já precisão é a proporção de verdadeiros positivos entre todos os positivos previstos. Uma precisão de 0.0000 sugere que o modelo não está conseguindo identificar nenhuma fraude corretamente. Ou seja, quando o modelo prevê fraude, ele nunca está correto.

O recall é a proporção de verdadeiros positivos entre todos os positivos reais. Um recall de 0.0000 indica que o modelo não está conseguindo identificar nenhum caso de fraude. Em outras palavras, o modelo falha completamente em detectar a classe minoritária.

O F1-Score é a média harmônica entre precisão e recall. Com precisão e recall ambos em 0, o F1-Score também é 0. Isso confirma que o modelo não está fazendo boas previsões para a classe minoritária.

A área sob a curva ROC (AUC-ROC) mede a capacidade do modelo de distinguir entre as classes. Um AUC-ROC de aproximadamente 0.5 indica que o modelo está se comportando como um classificador aleatório, não conseguindo distinguir efetivamente entre as classes.

In [13]:
# Definindo a faixa de valores para o Random Search
param_dist = {
    'hidden_layer_sizes': [(50, 50), (100,), (150, 100)],
    'activation': ['tanh', 'relu'],
    'solver': ['adam', 'sgd'],
    'alpha': uniform(0.0001, 0.01),
    'learning_rate': ['constant', 'adaptive'],
}

# Definindo o modelo básico
model = MLPClassifier(max_iter=100)

# Configurando o RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, n_jobs=-1, cv=3, scoring='f1', random_state=42)

# Executando o Random Search
random_search.fit(X_train, y_train)

# Obtendo as melhores configurações
best_model = random_search.best_estimator_

# Prevendo no conjunto de teste
y_pred_random = best_model.predict(X_test)
y_prob_random = best_model.predict_proba(X_test)[:, 1]

# Avaliando o modelo otimizado
accuracy_random = accuracy_score(y_test, y_pred_random)
precision_random = precision_score(y_test, y_pred_random)
recall_random = recall_score(y_test, y_pred_random)
f1_random = f1_score(y_test, y_pred_random)
roc_auc_random = roc_auc_score(y_test, y_prob_random)

print(f"Random Search - Accuracy: {accuracy_random:.4f}")
print(f"Random Search - Precision: {precision_random:.4f}")
print(f"Random Search - Recall: {recall_random:.4f}")
print(f"Random Search - F1-Score: {f1_random:.4f}")
print(f"Random Search - AUC-ROC: {roc_auc_random:.4f}")

Random Search - Accuracy: 0.9977
Random Search - Precision: 0.0000
Random Search - Recall: 0.0000
Random Search - F1-Score: 0.0000
Random Search - AUC-ROC: 0.4918


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Comparação entre modelos
No contexto de detecção de fraudes em cartões de crédito, foram avaliados três modelos diferentes: um modelo de rede neural simples (MLPClassifier), um modelo otimizado usando GridSearchCV e um modelo otimizado usando RandomizedSearchCV. Os principais objetivos foram avaliar a capacidade de cada modelo em identificar fraudes e otimizar o desempenho através do ajuste de hiperparâmetros.

Todos os modelos, incluindo o modelo de rede neural simples e as versões otimizadas com GridSearchCV e RandomizedSearchCV, enfrentam desafios semelhantes ao lidar com o desbalanceamento das classes. A alta acurácia observada em todos os casos é uma consequência do desbalanceamento dos dados, onde a classe majoritária domina as previsões.

Para melhorar a eficácia na detecção de fraudes em cartões de crédito, é crucial abordar o problema do desbalanceamento de dados, uma vez que modelos padrão tendem a ser tendenciosos em relação à classe majoritária. O rebalanceamento dos dados pode ser feito através de técnicas como oversampling da classe minoritária com SMOTE (Synthetic Minority Over-sampling Technique) ou undersampling da classe majoritária. Além disso, é essencial explorar modelos alternativos que lidam melhor com dados desbalanceados, como Random Forest e XGBoost, que possuem mecanismos internos de ponderação das classes e podem ajustar-se melhor às características dos dados. Também é importante ajustar as métricas de avaliação para refletir adequadamente a performance do modelo em contextos desbalanceados; o F1-Score e a curva Precision-Recall são mais indicados do que a acurácia, pois fornecem uma visão mais detalhada sobre a capacidade do modelo em identificar corretamente a classe minoritária, crucial para a detecção eficaz de fraudes.

# Visualizações  * INDO ALÉM *


## Figura 1
A figura 1 mostra uma curva ROC (Receiver Operating Characteristic), que é uma ferramenta útil para avaliar a performance dos modelos de classificação, especialmente em contextos de detecção de fraudes. A curva ROC ilustra a relação entre a taxa de verdadeiros positivos (TPR) e a taxa de falsos positivos (FPR), ajudando a identificar o trade-off entre sensibilidade e especificidade. Um modelo ideal teria uma curva que sobe rapidamente para o canto superior esquerdo, indicando alta TPR e baixa FPR.



In [20]:
# Calculando as métricas da curva ROC
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Criando um DataFrame para o gráfico
roc_df = pd.DataFrame({
    'FPR': fpr,
    'TPR': tpr
})

# Plotando a curva ROC
fig = px.line(roc_df, x='FPR', y='TPR', title=f'Curva ROC (AUC = {roc_auc:.4f})')
fig.update_layout(xaxis_title='Taxa de Falsos Positivos', yaxis_title='Taxa de Verdadeiros Positivos')
fig.show()


## Figura 2

Eixo X (horizontal): Representa a “Probabilidade de Previsão”, variando de 0.00 a 1.00. Isso indica a confiança do modelo nas suas previsões.
Eixo Y (vertical): Representa a “Frequência Absoluta”, que mostra quantas vezes cada valor de probabilidade foi observado.
Análise dos Resultados:

Barra Azul (Classe 0): A barra azul, que representa a classe 0, tem uma alta frequência de probabilidades próximas a 0. Isso sugere que o modelo está frequentemente confiante ao classificar transações como não fraudulentas.

Barra Vermelha (Classe 1): A barra vermelha, que representa a classe 1 (provavelmente fraudes), tem uma frequência muito baixa de probabilidades próximas a 1. Isso indica que o modelo raramente está confiante ao classificar transações como fraudulentas.

O gráfico mostra que o modelo tende a ser muito mais confiante ao identificar transações legítimas (classe 0) do que fraudes (classe 1). Isso pode ser um indicativo de que o modelo está enfrentando dificuldades para distinguir fraudes de transações legítimas, possivelmente devido ao desbalanceamento dos dados.

In [22]:
# Criando um DataFrame para o gráfico
hist_df = pd.DataFrame({
    'Probabilidade': y_pred_proba,
    'Classe': y_test
})

# Plotando o histograma das probabilidades
fig = px.histogram(hist_df, x='Probabilidade', color='Classe', barmode='overlay',
                   title='Histograma das Probabilidades de Previsão',
                   labels={'Probabilidade': 'Probabilidade de Previsão', 'Classe': 'Classe'})
fig.update_layout(xaxis_title='Probabilidade de Previsão', yaxis_title='Frequência')
fig.show()

